 # Using met.no forecasts with weather-tools



 This notebook-style script shows how to fetch met.no forecasts and combine them

 with SILO observations using the new `weather_tools` APIs. Open it directly in

 JupyterLab or VS Code with Jupytext to run the cells interactively.

 ## Prerequisites

 - Network access to `https://api.met.no` and a valid contact e-mail for the User-Agent header.

 - Set `SILO_API_KEY` (your registered SILO e-mail) or pass `api_key=` directly to `SiloAPI`.

 - Install `weather-tools` in your environment (either via `pip install weather-tools` or the local checkout).

 - Optional: enable logging (e.g. `logging.basicConfig(level=logging.INFO)`) to inspect requests.

In [1]:
import pandas as pd

from weather_tools.merge_weather_data import get_merge_summary, merge_historical_and_forecast
from weather_tools.metno_api import MetNoAPI
from weather_tools.metno_models import MetNoFormat, MetNoQuery
from weather_tools.silo_api import SiloAPI
from weather_tools.silo_models import AustralianCoordinates

In [2]:
# silo_api = SiloAPI(log_level="INFO")
# station_meta = silo_api.search_stations("Northam").iloc[0]

station_meta = {
    "station_code": 10111,
    "name": "NORTHAM",
    "latitude": -31.651,
    "longitude": 116.659,
    "state": "WA",
    "elevation": 170.0,
}

## 1. Configure the met.no API client

 met.no requires a descriptive User-Agent string that includes contact details.

 Replace the placeholder below with your own application name and e-mail.

In [3]:
api = MetNoAPI(
    # user_agent="weather-tools-example/0.1 (contact: you@example.com)",
    enable_cache=True,
    log_level="DEBUG",
)

## 2. Build a forecast query for your location

 The `AustralianCoordinates` model validates that the latitude and longitude sit

 within the range supported by SILO (GDA94 datum). Adjust the coordinates to

 target your site of interest.

In [4]:
coordinates = AustralianCoordinates(
    latitude=station_meta["latitude"], longitude=station_meta["longitude"]
)  # Northam, WA
query = MetNoQuery(coordinates=coordinates, format=MetNoFormat.COMPACT)

response = api.query_forecast(query)
metadata = response.get_meta()

metadata

[10/30/25 11:33:15] DEBUG    🌐 Constructed URL:                                                                   
                             https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=-31.651&lon=116.659

                    DEBUG    📋 User-Agent: weather-tools/0.0.0 (Python 3.12)

                    DEBUG    Making request (attempt 1/3):                                                         
                             https://api.met.no/weatherapi/locationforecast/2.0/compact

[10/30/25 11:33:16] DEBUG    Cached response for: e9450af0091badb1a028399707618845

                    DEBUG    Request successful on attempt 1

{'updated_at': '2025-10-30T01:17:02Z',
 'units': {'air_pressure_at_sea_level': 'hPa',
  'air_temperature': 'celsius',
  'cloud_area_fraction': '%',
  'precipitation_amount': 'mm',
  'relative_humidity': '%',
  'wind_from_direction': 'degrees',
  'wind_speed': 'm/s'}}

 ## 3. Convert forecasts to daily and hourly tables

 `MetNoAPI.to_dataframe` can aggregate the hourly GeoJSON payload to daily summaries

 or return hourly data. Use the `frequency` parameter to control aggregation:

 - `frequency='daily'` (default) - Daily aggregates

 - `frequency='hourly'` - Raw hourly data

 - `frequency='weekly'` or `frequency='monthly'` - Other time periods

In [5]:
daily_forecasts = api.to_dataframe(response, frequency="daily")
hourly_forecasts = api.to_dataframe(response, frequency="hourly")

daily_forecasts.head()

,date,min_temperature,max_temperature,total_precipitation,avg_wind_speed,max_wind_speed,avg_relative_humidity,avg_pressure,avg_cloud_fraction,dominant_weather_symbol
0,2025-10-30,9.9,21.8,0.0,3.400000,6.0,73.747619,1015.247619,25.376190,cloudy
1,2025-10-31,12.9,24.7,1.3,5.762500,7.7,64.866667,1013.108333,50.062500,rain
2,2025-11-01,11.0,22.2,0.0,5.226316,6.8,49.500000,1018.005263,29.768421,partlycloudy_day
3,2025-11-02,14.0,24.0,0.0,4.250000,4.9,36.525000,1022.000000,0.000000,clearsky_day
4,2025-11-03,17.0,28.3,0.6,4.525000,4.9,23.625000,1014.500000,62.700000,lightrain


In [6]:
hourly_forecasts.head()

,time,air_pressure_at_sea_level,air_temperature,cloud_area_fraction,relative_humidity,wind_from_direction,wind_speed,precipitation_amount,symbol_code
0,2025-10-30 03:00:00,1016.1,19.7,95.3,49.8,276.5,5.3,0.0,cloudy
1,2025-10-30 04:00:00,1015.7,20.5,89.1,46.0,280.0,5.3,0.0,cloudy
2,2025-10-30 05:00:00,1014.9,21.6,80.5,41.3,280.3,5.5,0.0,partlycloudy_day
3,2025-10-30 06:00:00,1014.5,21.8,92.2,42.5,282.0,6.0,0.0,cloudy
4,2025-10-30 07:00:00,1014.3,21.3,46.1,45.0,276.2,5.9,0.0,partlycloudy_day


 ### Quick helper: `get_daily_forecast`

 `get_daily_forecast` returns a pandas DataFrame directly with daily summaries,


In [7]:
daily_df = api.get_daily_forecast(
    latitude=coordinates.latitude,
    longitude=coordinates.longitude,
    days=5,
)

daily_df  # Already a DataFrame!

                    DEBUG    🌐 Constructed URL:                                                                   
                             https://api.met.no/weatherapi/locationforecast/2.0/compact?lat=-31.651&lon=116.659

                    DEBUG    📋 User-Agent: weather-tools/0.0.0 (Python 3.12)

                    DEBUG    Cache hit for request: e9450af0091badb1a028399707618845

,date,min_temperature,max_temperature,total_precipitation,avg_wind_speed,max_wind_speed,avg_relative_humidity,avg_pressure,avg_cloud_fraction,dominant_weather_symbol
0,2025-10-30,9.9,21.8,0.0,3.400000,6.0,73.747619,1015.247619,25.376190,cloudy
1,2025-10-31,12.9,24.7,1.3,5.762500,7.7,64.866667,1013.108333,50.062500,rain
2,2025-11-01,11.0,22.2,0.0,5.226316,6.8,49.500000,1018.005263,29.768421,partlycloudy_day
3,2025-11-02,14.0,24.0,0.0,4.250000,4.9,36.525000,1022.000000,0.000000,clearsky_day
4,2025-11-03,17.0,28.3,0.6,4.525000,4.9,23.625000,1014.500000,62.700000,lightrain


## 4. Merge met.no forecasts with SILO history

 Pull the last five days from the SILO DataDrill API for the same coordinates,

 then merge that history with the met.no forecast. The helper automatically

 converts column names, fills optional variables (when enabled), and annotates

 the data source for each record.

In [8]:
first_forecast_date = pd.to_datetime(daily_df["date"]).min()
# history_end = first_forecast_date - pd.Timedelta(days=1)
history_start = first_forecast_date - pd.Timedelta(days=4)
silo_api = SiloAPI(log_level="INFO")
silo_history = silo_api.get_gridded_data(
    latitude=coordinates.latitude,
    longitude=coordinates.longitude,
    start_date=history_start.strftime("%Y%m%d"),
    end_date=first_forecast_date.strftime("%Y%m%d"),
    variables=["rainfall", "max_temp", "min_temp", "evaporation"],
)

# silo_history = raw_silo.rename(columns={"YYYY-MM-DD": "date"}).copy()

# silo_history["date"] = pd.to_datetime(silo_history["date"])

# required_cols = ["date", "min_temp", "max_temp", "daily_rain"]
# missing = [col for col in required_cols if col not in silo_history.columns]
# if missing:
#     raise ValueError(f"Missing required columns from SILO response: {missing}")

# silo_data = silo_history[required_cols].sort_values("date").reset_index(drop=True)

merged = merge_historical_and_forecast(
    silo_data=silo_history,
    metno_data=daily_df,
    overlap_strategy="prefer_silo",
    return_cols="silo_only",
)

merged

[10/30/25 11:33:18] INFO     Auto-detected transition date: 2025-10-30 00:00:00

                    INFO     Merged 4 SILO records with 5 met.no records. Total: 9 records

,latitude,longitude,daily_rain,max_temp,min_temp,evap_pan,date,data_source,is_forecast
0,-31.651,116.659,0.0,24.0,7.9,6.8,2025-10-26,silo,False
1,-31.651,116.659,0.0,27.2,8.8,7.4,2025-10-27,silo,False
2,-31.651,116.659,0.0,35.0,9.8,8.5,2025-10-28,silo,False
3,-31.651,116.659,0.0,25.5,11.8,6.2,2025-10-29,silo,False
4,NaN,NaN,0.0,21.8,9.9,NaN,2025-10-30,metno,True
5,NaN,NaN,1.3,24.7,12.9,NaN,2025-10-31,metno,True
6,NaN,NaN,0.0,22.2,11.0,NaN,2025-11-01,metno,True
7,NaN,NaN,0.0,24.0,14.0,NaN,2025-11-02,metno,True
8,NaN,NaN,0.6,28.3,17.0,NaN,2025-11-03,metno,True


In [9]:
merge_summary = get_merge_summary(merged)
merge_summary

{'total_records': 9,
 'silo_records': np.int64(4),
 'metno_records': np.int64(5),
 'date_range': {'start': Timestamp('2025-10-26 00:00:00'),
  'end': Timestamp('2025-11-03 00:00:00'),
  'days': 9},
 'silo_period': {'start': Timestamp('2025-10-26 00:00:00'),
  'end': Timestamp('2025-10-29 00:00:00')},
 'metno_period': {'start': Timestamp('2025-10-30 00:00:00'),
  'end': Timestamp('2025-11-03 00:00:00')},
 'transition_date': Timestamp('2025-10-30 00:00:00')}

## 4b. Merge Forecast with SILO PatchedPoint 

 Pull the last four days from the SILO Patch Point API for the same coordinates, using the station code. 


In [10]:
first_forecast_date = pd.to_datetime(daily_forecasts["date"]).min()
history_end = first_forecast_date  # - pd.Timedelta(days=1) # Include day before forecast to check overlap handling
history_start = history_end - pd.Timedelta(days=4)


silo_station_data = silo_api.get_station_data(
    station_code=str(station_meta["station_code"]),
    start_date=history_start.strftime("%Y%m%d"),
    end_date=history_end.strftime("%Y%m%d"),
    variables=["rainfall", "max_temp", "min_temp", "evaporation"],
)

merge_historical_and_forecast(
    silo_data=silo_station_data,
    metno_data=daily_df,
    return_cols="silo_only",
)

[10/30/25 11:33:20] INFO     Auto-detected transition date: 2025-10-30 00:00:00

                    INFO     Merged 4 SILO records with 5 met.no records. Total: 9 records

,station,daily_rain,max_temp,min_temp,evap_pan,date,data_source,is_forecast
0,10111.0,0.0,24.0,8.0,6.8,2025-10-26,silo,False
1,10111.0,0.0,27.0,9.0,7.4,2025-10-27,silo,False
2,10111.0,0.0,35.0,9.6,8.5,2025-10-28,silo,False
3,10111.0,0.0,25.6,12.2,0.0,2025-10-29,silo,False
4,NaN,0.0,21.8,9.9,NaN,2025-10-30,metno,True
5,NaN,1.3,24.7,12.9,NaN,2025-10-31,metno,True
6,NaN,0.0,22.2,11.0,NaN,2025-11-01,metno,True
7,NaN,0.0,24.0,14.0,NaN,2025-11-02,metno,True
8,NaN,0.6,28.3,17.0,NaN,2025-11-03,metno,True
